In [31]:
#imported modules
import numpy as np
import torch as torch
import torchvision.transforms as transforms
import gc
from torch.autograd import Variable
from models.small_cnn import SmallCNN
from models.wideresnet import WideResNet
from torchvision import transforms,datasets
from models.wideresnet import WideResNet
from torch.autograd import Variable

#remove any garbage data from the CUDA cores
gc.collect()

#defines the function for the FGSM attack
def fgsm_attack(inputImage, epsilon, gradient) :
    #check if the gradient is a list or a float value
    if type(gradient) is not float :
        #get the perturbed image
        perturbedImage = inputImage + epsilon*gradient.sign()
    else :
        #get the perturbed image with the precalculated sign gradient
        perturbedImage = inputImage + epsilon*gradient
    #END IF

    #clamp the data to a range between zero and one
    perturbedImage = torch.clamp(perturbedImage, 0, 1)

    #return the perturbed image
    return perturbedImage
#END fgsm_attack

#initializes an FGSM attack
def startFGSM(file, save, epsilons, mode) :
    #empty the CUDA cache
    torch.cuda.empty_cache()

    #defines the shape of tensor testing sets
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.0,), (1.0,))])
    
    #set the working device to the CUDA cores
    device = torch.device("cuda")

    #loads the images for the testing dataset into a numpy array
    images = np.load(file)

    #mode 1 is MNIST and mode 2 is CIFAR10
    if mode == 1 :
        #loads the dataset for the MNIST data
        testingSet = datasets.MNIST(root = './data', train = False, transform = transform, download = True)
        
        #gets the neural network model used for the training
        model = SmallCNN().to(device)
        model.load_state_dict(torch.load('./checkpoints/model_mnist_smallcnn.pt'))
        
        #loads the MNIST dataset into a tensor
        testingLoader = torch.utils.data.DataLoader(testingSet, batch_size = 1, shuffle = False)

        #set the network to evaluation mode
        model.eval()

        #set the loss function
        loss = torch.nn.functional

        #for each image in the dataset
        for i, (data, target) in enumerate(testingLoader) : 
            #store the data in the GPU memory
            data, target = data.to("cuda"), target.to("cuda")
            data.requires_grad = True
            output = model(data)

            #gets the initial prediction for the item
            initial = output.max(1, keepdim = True)[1] 

            #if the item is equal to the target item
            if initial.item() == target.item():
                #calculate the loss value
                lossValue = loss.nll_loss(output, target)
                model.zero_grad()
                lossValue.backward()

                #get the gradient
                data_grad = data.grad.data

                #get the perturbed image
                perturbed_data = fgsm_attack(data,epsilons, data_grad)

                #get the output of the image in the network
                output = model(perturbed_data)

                #convert the image back to a numpy array
                newImage = perturbed_data.squeeze().detach().cpu().numpy()

                #replace the image with the new perturbed image
                images[i] = newImage
            #END IF
        #END FOR
    elif mode == 2 :
        #load the dataset for the CIFAR10 data
        testingSet = datasets.CIFAR10(root = './data', train=False, transform = transform, download=True)
        
        #gets the neural network model used for the training
        model = WideResNet().to(device)
        model.load_state_dict(torch.load('./checkpoints/model_cifar_wrn.pt'))
        
        #loads the CIFAR dataset into a tensor
        testingLoader = torch.utils.data.DataLoader(testingSet, batch_size = 1, shuffle = False)

        #for each image in the dataset
        for i, (data, target) in enumerate(testingLoader) :
            #store the data in the GPU memory
            data, target = data.to("cuda"), target.to("cuda")

            #gets the image and the image through the network
            input = Variable(data.data, requires_grad = True)
            output = model(input)

            #gets the criterion values
            criterion = torch.nn.CrossEntropyLoss()
            cost = criterion(output, target)

            #count the gradients
            model.zero_grad()
            cost.backward()

            #gets the adverserial image
            newImage = fgsm_attack(input, input.grad.detach().sign(), epsilons)

            #set the new perturbed image into the images array
            images[i] = newImage.squeeze().detach().cpu().numpy().transpose((1, 2, 0))
        #END FOR
    #END IF
    
    #save the purturbed images
    np.save(save, images)
#END startFGSM

startFGSM("./data_attack/mnist_X.npy", "./data_attack/mnist_X_adv.npy", 0.2990, 1)
startFGSM("./data_attack/cifar10_X.npy", "./data_attack/cifar10_X_adv.npy", 0.0309, 2)

Files already downloaded and verified
